## LZify: Applying Universal Prediction to Musical Style

This notebook demonstrates the use of Incremental Parsing (IP) from the LZ78 method for creating a dictionary of motifs. These motifs are later used to generate new sequences resembling the input sequence.


The IPMotif function computes the motif dictionary discovered in the text. It uses Incremental Parsing method to parse the text into unseen motifs.

In [1]:
def IPMotif(text):
    """Compute an associative dictionary (the motif dictionary)."""
    
    dictionary = {} 
    motif = ""
    result = []
    for c in text:
        motif = motif + c
        if motif in dictionary:
            # Increase count for existing motif
#            print '%s in dictionary' % motif
            dictionary[motif] += 1
        else:
            # Add motif to the dictionary.
            dictionary[motif] = 1
            motif = ""
#            print 'add %s to dictionary' % motif

    return dictionary


In [32]:
#text = "Cest impressionnant le TGV en quatre heures, on traverse toute la France. On na pas le temps de regarder les paysages. Cest fantastique mais je reconnais que jai un peu la nostalgie des vieux trains. Dans les vieux films, jaime le bruit particulier que les roues du train font sur les rails. Cest un bruit que jassocie au cinéma, ça donne du rythme aux films policiers. Quand on entend ce bruit, ça signifie quil y a un danger. En TGV, je me sens comme dans un avion. Il faut reconnaître que cest confortable. Jen profite pour lire tranquillement le programme du"
#text = 'abracadabra' #bracadabra'
text = "By using music written in a certain style as training data, parameters can becalculated for Markov chains and hidden Markov models to capture the musical style of the training data as mathematical models."
dict1 = IPMotif(text)
print dict1

{' c': 3, ' a': 1, ' f': 1, ' d': 1, ' h': 1, ' m': 3, ' s': 1, ' p': 1, ' w': 1, ' t': 2, ' ': 16, 'ty': 1, 'tu': 1, 'tr': 1, 'th': 1, 'ta': 2, 'e o': 1, 'e m': 1, 'yl': 1, 'de': 2, 'da': 1, 'e t': 1, 'n b': 1, 'ng ': 1, 'd': 4, 'h': 1, 'ers': 1, 'l': 1, 'n M': 1, 'p': 1, 'n ': 3, 'em': 1, 'el': 1, 'ec': 1, 'et': 1, 'ark': 2, 'er': 2, ' ch': 1, ' ca': 1, 'o ': 1, 're': 1, 'r': 4, 'ra': 1, 't': 7, 'cal': 2, 'e ': 4, 'c': 4, 'ate': 1, 'g': 1, 'od': 1, 'o': 4, 'ati': 1, 's': 6, 'r ': 1, 'ov': 1, ' ma': 1, 's a': 1, 'ca': 3, ' mo': 1, 's t': 1, 'usi': 1, 'B': 1, 'al': 1, ' th': 1, 'f': 1, 'us': 2, 'n': 8, 'ul': 1, 'ain': 1, 'del': 1, 'v': 1, 'cal ': 1, 'at': 3, 'ai': 2, 'am': 1, 'it': 1, 'as': 1, 'ar': 3, 'ini': 1, 'in': 3, 'ic': 1, 'id': 1, 'ni': 1, 'arko': 1, 'nd': 1, 'ng': 2, 's.': 1, 's ': 3, 'M': 1, 'a,': 1, 'ta ': 1, 'in ': 1, 'a': 13, 'e': 11, 'i': 7, 'm': 1, 'st': 1, 'u': 4, 'y': 2}


The IPContinuation function transforms the IPMotif dictionary into a tree-like representation to allow finding continuations for new  motifs.

In [33]:
def IPContinuation(dict1):
    """Compute continuation dictionary from a motif dictionary"""
    
    dict2 = {}
    for Wk in dict1:
        counter = dict1[Wk]
        W = Wk[:-1]
        #print 'W =',W
        k = Wk[-1]
        #print 'k-',k
        if W in dict2:
            dict2[W].append((k,counter))
            #print 'dict2', dict2
        else:
            dict2[W] = [(k,counter)]
            #print 'dict2', dict2
    dict2 = Normalize(dict2)    
    return dict2
                    
                    
def Normalize(dict2):
    """Turns the counters in every element of dict2 to probabilities"""
    
    for W in dict2:
        cnt = [tup[1] for tup in dict2[W]]
        ttl = sum(cnt)
        for k,tup in enumerate(dict2[W]):
            dict2[W][k] = (tup[0],float(tup[1])/ttl)
    return dict2

In [34]:
dict2 = IPContinuation(dict1)
print dict2

{'': [(' ', 0.16161616161616163), ('d', 0.04040404040404041), ('h', 0.010101010101010102), ('l', 0.010101010101010102), ('p', 0.010101010101010102), ('r', 0.04040404040404041), ('t', 0.0707070707070707), ('c', 0.04040404040404041), ('g', 0.010101010101010102), ('o', 0.04040404040404041), ('s', 0.06060606060606061), ('B', 0.010101010101010102), ('f', 0.010101010101010102), ('n', 0.08080808080808081), ('v', 0.010101010101010102), ('M', 0.010101010101010102), ('a', 0.13131313131313133), ('e', 0.1111111111111111), ('i', 0.0707070707070707), ('m', 0.010101010101010102), ('u', 0.04040404040404041), ('y', 0.020202020202020204)], ' c': [('h', 0.5), ('a', 0.5)], 'ai': [('n', 1.0)], 'ca': [('l', 1.0)], ' m': [('a', 0.5), ('o', 0.5)], 'ar': [('k', 1.0)], 'us': [('i', 1.0)], ' t': [('h', 1.0)], 'in': [('i', 0.5), (' ', 0.5)], 'ark': [('o', 1.0)], 'er': [('s', 1.0)], ' ': [('c', 0.2), ('a', 0.06666666666666667), ('f', 0.06666666666666667), ('d', 0.06666666666666667), ('h', 0.06666666666666667), ('m

Generting new sequence is done by traversing the IPContinuation tree and selecting possible branches according to their weights. If motif is not found, its last symbol is removed and the process is repeated for a shorter motif.

In [35]:
from numpy.random import multinomial as randm
from numpy import where

def IPGenerate(n,dict2):
    p = 0
    out = ""
    for k in range(n):
        while True:
            context = out[-p:]
            if context in dict2:
                prob = [tup[1] for tup in dict2[context]]
                conti = where(randm(1,prob))[0][0]
                cont = dict2[context][conti][0]
                out = out + cont
#                print prob, conti, cont, out
                break
            else:
                p = p-1
    return out

In [36]:
out = IPGenerate(200,dict2)
print out

delodelndela, a,e mo wamn bng mamtremecal ainidelarkovMral mo ain Me ovatinical deldelta htyltyln M moda, cal movtusinininical ho mo wers arkodarko atical pcal pusinitylta mo ha,Me o we thdelng wvcal 


Try this with different languages. Can you identify which was the original language? Or distinguish between which languages were used for training?
These simple experiments demonstrate few important points:
1. The method captures the "texture" of the language but not it's meaning
2. We could parse a new text using IPMotifs from two languages, count the length and number of motifs in order to decide what was the language of the new text
3. In order to use it with musical information, we need first to translate audio to features, or in case of polyphonic midi change this into some proper representation. One possibility is using virtual fundamental or chroma for harmony, or some other speciailzed representation to capture repetition in terms of other specific musical properties

In case when the representation (features) are partial or "lossy", the reconstruction is done by keeping a pointer to the original data

# Comparing to Markov
Let's create a similar continuation dictionary using Markov approach.

We compute Markov model by setting a fixed length motif (Markov memory size) and finding all possible continuation for that motif.


In [37]:
def Markov(text,order=0):
    """Compute a Markov models (fixed length motif dictionary)."""
    
    dict3 = {}
    for i in range(len(text)-order):
        W = text[i:i+order]
        #print 'W', W
        k  = text[i+order]
        #print 'k', k
        if W in dict3:
            if k in list(zip(*dict3[W])[0]):
                dict3[W][k] += 1
                print 'dict', dict3
            else:
                dict3[W][k] = 1
                print 'dict', dict3
        else:
            dict3[W] = {k:1}
            print 'dict', dict3

    for x in dict3:
        dict3[x] = dict3[x].items()
    dict3 = Normalize(dict3)    
    return dict3


In [38]:
text1 = 'abracadabra'
Mdict = Markov(text1,2)
print Mdict

dict {'ab': {'r': 1}}
dict {'ab': {'r': 1}, 'br': {'a': 1}}
dict {'ab': {'r': 1}, 'ra': {'c': 1}, 'br': {'a': 1}}
dict {'ac': {'a': 1}, 'ab': {'r': 1}, 'ra': {'c': 1}, 'br': {'a': 1}}
dict {'ac': {'a': 1}, 'ab': {'r': 1}, 'ca': {'d': 1}, 'ra': {'c': 1}, 'br': {'a': 1}}
dict {'ac': {'a': 1}, 'ab': {'r': 1}, 'ad': {'a': 1}, 'ca': {'d': 1}, 'ra': {'c': 1}, 'br': {'a': 1}}
dict {'ac': {'a': 1}, 'ab': {'r': 1}, 'ad': {'a': 1}, 'ca': {'d': 1}, 'da': {'b': 1}, 'ra': {'c': 1}, 'br': {'a': 1}}
dict {'ac': {'a': 1}, 'ab': {'r': 2}, 'ad': {'a': 1}, 'ca': {'d': 1}, 'da': {'b': 1}, 'ra': {'c': 1}, 'br': {'a': 1}}
dict {'ac': {'a': 1}, 'ab': {'r': 2}, 'ad': {'a': 1}, 'ca': {'d': 1}, 'da': {'b': 1}, 'ra': {'c': 1}, 'br': {'a': 2}}
{'ac': [('a', 1.0)], 'ab': [('r', 1.0)], 'ad': [('a', 1.0)], 'ca': [('d', 1.0)], 'da': [('b', 1.0)], 'ra': [('c', 1.0)], 'br': [('a', 1.0)]}


We can start experimenting directly with this model, but would need to set up some random initial motif to start the IPContinuation process. Another option is to include all models from order 0 to the chosen maximal Markov order.

In [39]:
def MaxOrder(text,order):
    dictall = {}
    for o in range(order):
        dicto = Markov(text,o)
        dictall.update(dicto)
    return dictall

In [40]:
text1 = 'abracadabra'
dictM1 = MaxOrder(text1,5)
print dictM1

dict {'': {'a': 1}}
dict {'': {'a': 1, 'b': 1}}
dict {'': {'a': 1, 'r': 1, 'b': 1}}
dict {'': {'a': 2, 'r': 1, 'b': 1}}
dict {'': {'a': 2, 'r': 1, 'b': 1, 'c': 1}}
dict {'': {'a': 3, 'r': 1, 'b': 1, 'c': 1}}
dict {'': {'a': 3, 'r': 1, 'b': 1, 'c': 1, 'd': 1}}
dict {'': {'a': 4, 'r': 1, 'b': 1, 'c': 1, 'd': 1}}
dict {'': {'a': 4, 'r': 1, 'b': 2, 'c': 1, 'd': 1}}
dict {'': {'a': 4, 'r': 2, 'b': 2, 'c': 1, 'd': 1}}
dict {'': {'a': 5, 'r': 2, 'b': 2, 'c': 1, 'd': 1}}
dict {'a': {'b': 1}}
dict {'a': {'b': 1}, 'b': {'r': 1}}
dict {'a': {'b': 1}, 'r': {'a': 1}, 'b': {'r': 1}}
dict {'a': {'c': 1, 'b': 1}, 'r': {'a': 1}, 'b': {'r': 1}}
dict {'a': {'c': 1, 'b': 1}, 'r': {'a': 1}, 'b': {'r': 1}, 'c': {'a': 1}}
dict {'a': {'c': 1, 'b': 1, 'd': 1}, 'r': {'a': 1}, 'b': {'r': 1}, 'c': {'a': 1}}
dict {'a': {'c': 1, 'b': 1, 'd': 1}, 'r': {'a': 1}, 'b': {'r': 1}, 'c': {'a': 1}, 'd': {'a': 1}}
dict {'a': {'c': 1, 'b': 2, 'd': 1}, 'r': {'a': 1}, 'b': {'r': 1}, 'c': {'a': 1}, 'd': {'a': 1}}
dict {'a': {'c'

In [41]:
out = IPGenerate(100,dictM1)
print out

racadabracadabracadabracadabracadabracadabracadabracadabracadabracadabracadabracadabracadabracadabra


Experiment: try using 'abracadabra' with IPGenerate. 

* At what order the generated sequence becomes a simple loop?
* What happens in the Markov model that creates the loop? Look at the combined Markov dictionary.

Try generating sequences using Markov order less then maximum order. Compare this to IPContinuation.

Try it on longer text (text1 below for example). What order should you choose? Try it on music (how?).
* Which one is more "natural"? Discuss your findings.
* Compare the sizes of the dictionaries (len(dict2.keys()))

In text it seems that Markov captures better the words, while LZ stays on the syllable level. 

In [42]:
#text1 = "Exhale le vertige, et les danseurs prudents Ne contempleront pas sans d'amres nauses Le sourire \xc3\xa9ternel de tes trente-deux dents. Pourtant, qui n'a serr\xc3\xa9 dans ses bras un squelette, Et qui ne s'est nourri des choses du tombeau ? Qu'importe le parfum, l'habit ou la toilette ? Qui fait le d\xc3\xa9go\xc3\xbbt\xc3\xa9 montre qu'il se croit beau. Bayad\xc3\xa8re sans nez, irr\xc3\xa9sistible gouge, Dis donc \xc3\xa0 ces danseurs qui font les offusqu\xc3\xa9s : Fiers mignons, malgr\xc3\xa9 l'art des poudres et du rouge,Vous sentez tous la mort ! \xc3\x94 squelettes musqu\xc3\xa9s,Antino\xc3\xbcs fl\xc3\xa9tris, dandys, \xc3\xa0 face glabre, Cadavres verniss\xc3\xa9s, lovelaces chenus, Le branle universel de la danse macabre Vous entra\xc3\xaene en des lieux qui ne sont pas connus !Des quais froids de la Seine aux bords br\xc3\xbblants du Gange, Le troupeau mortel saute et se p\xc3\xa2me, sans voir Dans un trou du plafond la trompette de l'Ange Sinistrement b\xc3\xa9ante ainsi qu'un tromblon noir. En tout climat, sous tout soleil, la Mort t'admire En tes contorsions, risible Humanit\xc3\xa9, Et souvent, comme toi, se parfumant de myrrhe, M\xc3\xaale son ironie \xc3\xa0 ton insanit\xc3\xa9 ! "
text1 = text

dict1 = IPMotif(text1)
#print dict1    

dict2 = IPContinuation(dict1)
#print dict2

outLZ = IPGenerate(700,dict2)
print 'LZ Generate'
print outLZ

print '========'

dict3 = MaxOrder(text1,3)
outM = IPGenerate(700,dict3)
print 'Markov Order 3 Generate'
print outM

LZ Generate
B mo del chng thmcal cal aters thin Mr s aters.Mtusin blng s al ch th cal markodelarko cal w movema,ta png mainical cal pta mo fMcal cal chain bo s. pcal fers thda,alodelraldarkovethta thatityl hdelharkodelndarkovltylov hn b chls alain Markovloda,yls.idelalusitheth cal thvr materstyldelta cal h chamng mov main barkodeln Mov delecal mo arkodelemle th fvusical wMn Mr fo chs thratidelatitre mo win Maltr fn M delpn b ain Mg arkovers.e ovuln M hrarkodelcal ths ainical pdelo fa,arkodastyl thin Mpusidelatetylin bpta movs.ta cal pers.cal the th mo modatecal ma,ers. matemidels atitre odelBodarkodelthre main bvv marko mamg hn bBpodel harkodelndain Mstr tho mas arkodelarko atecal chmarkodel s arkovulm 
dict {'': {'B': 1}}
dict {'': {'y': 1, 'B': 1}}
dict {'': {'y': 1, ' ': 1, 'B': 1}}
dict {'': {'y': 1, ' ': 1, 'B': 1, 'u': 1}}
dict {'': {'y': 1, ' ': 1, 's': 1, 'B': 1, 'u': 1}}
dict {'': {' ': 1, 'B': 1, 'i': 1, 's': 1, 'u': 1, 'y': 1}}
dict {'': {' ': 1, 'B': 1, 'i': 1, 'n': 1, 's':

In [55]:
print len(dict2.keys())
print len(dict3.keys())

32
131


In [56]:
print dict2.keys()

['', ' c', 'ai', 'ca', ' m', 'ar', 'us', ' t', 'in', 'ark', 'er', ' ', 'ng', 's ', 'ta', 'de', 'e ', 'a', 'c', 'e', 'd', 'n ', 'o', 'n', 'i', 's', 'r', 'u', 't', 'cal', 'y', 'at']


The LZ dictionary is smaller in size and it captures motifs (keys) of length 3 from text1 example. It is difficult to judge the quality of the generated sequence, and finding relations between data analysis and musical perception or aesthetics is still an open formidable question.